Tested on Python 3.9.0

You will have an issue with gensim if you just pip install gensim and try to run it. 
You will get this error:  ImportError: cannot import name 'triu' from 'scipy.linalg' - Gensim
1. pip install gensim 
2. Go into  <python-install>/site-packages/gensim/matutils.py & line 20 & 21 to this
    line 20:from numpy import triu
    line 21:from scipy.linalg import get_blas_funcs
3. Save file 
4. Restart Notebook & Run


In [1]:
import gensim
import scipy
print("Gensim version:", gensim.__version__)
print("SciPy version:", scipy.__version__)


Gensim version: 4.3.2
SciPy version: 1.13.0


In [2]:
import networkx as nx
import numpy as np
from gensim.models import Word2Vec

class Node2Vec:
    def __init__(self, graph, dimensions=64, walk_length=30, num_walks=200, p=1, q=1):
        self.graph = graph
        self.dimensions = dimensions
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.p = p  # Return hyperparameter
        self.q = q  # In-out hyperparameter

    def node2vec_walk(self, start_node):
        """ Simulate a random walk starting from start node. """
        walk = [start_node]
        while len(walk) < self.walk_length:
            current = walk[-1]
            neighbors = list(self.graph.neighbors(current))
            if not neighbors:
                break
            next_node = np.random.choice(neighbors)  # Random neighbor
            walk.append(next_node)
        return walk

    def simulate_walks(self):
        """ Repeatedly simulate random walks from each node. """
        walks = []
        for _ in range(self.num_walks):
            for node in self.graph.nodes():
                walks.append(self.node2vec_walk(node))
        return walks

    def learn_embeddings(self, walks):
        """ Learn embeddings by optimizing the skip-gram objective. """
        walks = [[str(node) for node in walk] for walk in walks]  # Convert all nodes to string
        model = Word2Vec(walks, vector_size=self.dimensions, window=10, min_count=1, sg=1)
        return model

# Example usage
G = nx.fast_gnp_random_graph(n=100, p=0.5)  # Generate a random graph
node2vec = Node2Vec(G)
walks = node2vec.simulate_walks()
model = node2vec.learn_embeddings(walks)

# Get vector for a node
node_id = 1
vector = model.wv[str(node_id)]
print(f"Vector for node {node_id}: {vector}")


Vector for node 1: [-0.14550367 -0.15498161  0.1717724   0.06062433 -0.31041783 -0.13868406
  0.0966089  -0.0961268  -0.21798424  0.03144229  0.01135332 -0.18190156
 -0.13287215 -0.0314305   0.03153683 -0.01961424  0.15545198 -0.05339472
 -0.11121581 -0.07837597  0.28068274  0.05381537  0.00666765  0.0278433
  0.07881247 -0.01566867 -0.09530184  0.07485341  0.12144967 -0.17363213
  0.00900874  0.02245182 -0.09925427 -0.17090663 -0.02475159  0.06749591
  0.04877013 -0.09814671  0.06321061 -0.05404817  0.020719   -0.07785162
 -0.02120511 -0.23796003  0.08355661  0.11032894  0.0987753  -0.042601
  0.1625481  -0.04584011 -0.10188025  0.22827905 -0.04897941 -0.05289655
  0.16793558  0.13725983  0.19852613 -0.04462974  0.11113588  0.25013354
  0.17508428 -0.23302913 -0.09113725 -0.08907607]


In [3]:
import networkx as nx
import random

def random_walk(G, start_node, walk_length):
    walk = [start_node]
    while len(walk) < walk_length:
        current = walk[-1]
        neighbors = list(G.neighbors(current))
        if len(neighbors) > 0:
            walk.append(random.choice(neighbors))
        else:
            break
    return walk


In [4]:
def generate_walks(G, num_walks, walk_length):
    walks = []
    nodes = list(G.nodes())
    for _ in range(num_walks):
        random.shuffle(nodes)
        for node in nodes:
            walks.append(random_walk(G, node, walk_length))
    return walks


pip install gensim

In [5]:
from gensim.models import Word2Vec

def learn_embeddings(walks, dimensions=64, window_size=10, workers=4, iterations=20):
    walks = [list(map(str, walk)) for walk in walks]  # Convert node IDs to string
    model = Word2Vec(walks, vector_size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, epochs=iterations)
    return model

In [6]:
# Create a graph
G = nx.fast_gnp_random_graph(n=100, p=0.5)

# Generate walks
walks = generate_walks(G, num_walks=10, walk_length=80)

# Learn embeddings
model = learn_embeddings(walks)

# Get vector for a node
node_id = 1
vector = model.wv[str(node_id)]
print(f"Embedding for node {node_id}: {vector}")


Embedding for node 1: [-0.19940229 -0.13005815  0.05669034 -0.13285561  0.00896109 -0.15733446
 -0.00669526 -0.15037434 -0.02935633  0.00840888  0.04003152 -0.09181
 -0.12342535 -0.17020129  0.01531873  0.20348106 -0.00679581  0.09355938
  0.16271292  0.1204627   0.11511598  0.3202359   0.04831364  0.08045028
  0.02352599  0.03393415 -0.1157303  -0.04854004  0.13151301  0.02997072
 -0.05866247  0.03996886 -0.01753443 -0.04977854 -0.18731758  0.01409035
  0.12826896 -0.00926292  0.2363801   0.11029468  0.09326193 -0.13944273
 -0.07624269 -0.15035807  0.03261533 -0.21407004  0.09839049  0.08408295
  0.16969588  0.10679232 -0.12905364 -0.09566238  0.06378093  0.19733562
  0.10591678  0.21329893 -0.08753596 -0.07784798 -0.1961368   0.25115916
  0.17384723  0.02938189  0.11579252 -0.05280943]


In [7]:
from sklearn.manifold import TSNE
# Or: from umap import UMAP

def reduce_dimensions(embeddings, n_components=2):
    tsne = TSNE(n_components=n_components)
    # Or: umap = UMAP(n_components=n_components)
    reduced_embeddings = tsne.fit_transform(embeddings)
    return reduced_embeddings


In [8]:
import matplotlib.pyplot as plt

def plot_embeddings(embeddings, labels=None):
    plt.figure(figsize=(10, 8))
    plt.scatter(embeddings[:, 0], embeddings[:, 1], c=labels, cmap='viridis')
    plt.colorbar()
    plt.title('Graph Embeddings Visualization')
    plt.xlabel('Dimension 1')
    plt.ylabel('Dimension 2')
    plt.show()


In [9]:
# Assuming you have already learned embeddings and have them stored in a variable named 'embeddings'
# You can reduce the dimensions
reduced_embeddings = reduce_dimensions(embeddings)

# Plot the reduced embeddings
plot_embeddings(reduced_embeddings)
# Assuming 'labels' is a list or array of node labels
plot_embeddings(reduced_embeddings, labels)


NameError: name 'embeddings' is not defined